In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import sys
import os
import shutil
from PIL import Image, ImageFont, ImageDraw
import cv2

# 添加路径，以能正常导入mbsh、trainer
sys.path.append(r'..\Lib\trainer')

from mbsh.core.yolo import YOLO
import numpy as np
import pandas as pd
import glob

2022-11-08 08:58:41,892 - INFO - __init__.py - init_logger - 58 - ini  logger file D:\projects\Vision\Lib\logs\kernel-35b3e046-9ab6-4bd3-a3a4-136921395637.log


use temp dir:d:\znyx\temp


2022-11-08 08:59:08,523 - DEBUG - tpu_cluster_resolver.py - <module> - 32 - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2022-11-08 08:59:13,245 - DEBUG - __init__.py - <module> - 47 - Creating converter from 7 to 5
2022-11-08 08:59:13,246 - DEBUG - __init__.py - <module> - 47 - Creating converter from 5 to 7
2022-11-08 08:59:13,246 - DEBUG - __init__.py - <module> - 47 - Creating converter from 7 to 5
2022-11-08 08:59:13,247 - DEBUG - __init__.py - <module> - 47 - Creating converter from 5 to 7
2022-11-08 08:59:33,428 - INFO - utils.py - _init_num_threads - 147 - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-11-08 08:59:33,429 - INFO - utils.py - _init_num_threads - 159 - NumExpr defaulting to 8 threads.


In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

### 公共函数

In [3]:
delta1 = 4       # 用于合并间断帧
delta2 = 2 * 18  # 用于筛选最终息肉
fps = 18

# 统计一个视频模型标记息肉开始结束帧
def stat_ct_file(file_path):
    ct, start, end = 0, 0, 0
    results = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            arr = line.strip().split(',')
            if arr[1] == '1':
                if start == 0:
                    start = int(arr[0][:-4])
            else:
                if start != 0:
                    end = int(arr[0][:-4])
                    if end - start >= delta1:
                        results.append([start, end])
                    start = 0
                    
        return results


def stat_ct(data):
    ct, start, end = 0, 0, 0
    results = []
    for arr in data:
        if arr[1] == 1:
            if start == 0:
                start = int(arr[0][:-4])
        else:
            if start != 0:
                end = int(arr[0][:-4])
                if end - start >= delta1:
                    results.append([start, end])
#                 results.append([start, end])
                start = 0
                    
    return results


# 合并一个视频模型标记息肉开始结束帧
def merge_ct(arr):
    if len(arr) == 0:
        return []
    
    results = [arr[0]]
    for idx in range(1, len(arr)):
        if arr[idx][0] - results[-1][1]  < delta1:
            results[-1][1] = arr[idx][1]
        else:
            results.append(arr[idx])
            
    return results

# 按照长度过滤息肉，>= 2S的才选择
def filt_polyp(arr):
    results = []
    for start, end in arr:
        if end - start >= delta1 - 1:
            results.append([start, end])
    
    return results
    

# 将目录的图片名称按照名称数字从小到大排序
def get_dir_sorted_imgs(dirs):
    names = []
    if isinstance(dirs, str):
        if os.path.exists(dirs):
            names = [int(filename[:-4]) for filename in os.listdir(dirs) if filename.endswith('.jpg')]
    elif isinstance(dirs, list):
        for dir in dirs:
            if os.path.exists(dir):
                names.extend([int(filename[:-4]) for filename in os.listdir(dir) if filename.endswith('.jpg')])
    names = sorted(names)
    img_names = ['%s.jpg' % name for name in names]
    return img_names


### 息肉检测模型

In [4]:
model_path = r'D:\projects\Vision\polyp\Models\trained_weights_final.h5'
anchors_path = r'D:\projects\Vision\polyp\Models\yolo_anchors_0311.txt'
classes_path = r'D:\projects\Vision\polyp\Models\polyp_classes.txt'
iou = 0.1

model_image_size = (352, 352)
gpu_num = 1 

In [5]:
# 模型检测标记息肉
def detect_polyp_by_model(input_path, target_path, score):
    """
    input_path：输入视频图片目录
    target_path：输出标记图片目录
    score：模型参数
    """
    if not os.path.exists(target_path):
        os.mkdir(target_path)

    # 初始化模型
    yolo = YOLO(model_path=model_path, anchors_path=anchors_path, classes_path=classes_path,
               score=score, iou=iou, model_image_size=model_image_size, gpu_num=gpu_num)

    # 模型预测
    results = []
    img_names = get_dir_sorted_imgs(input_path)
    for filename in img_names:
        img_path = os.path.join(input_path, filename)
        pred_results, image, c = yolo.predict_file(img_path, target_path, draw_rect=True, cut=False, 
                                                         one_box=True, timing=False, expand=1)
        if len(pred_results) == 1:
            #results.append('%s,%s,%s,%s,%s,%s,%s\n' % (filename, 1, pred_results[0][4], pred_results[0][0], pred_results[0][1],pred_results[0][2], pred_results[0][3]))
            results.append((filename, 1))
        else:
            results.append((filename, 0))

    return results

# 息肉统计结果写入文件
def write_stat_result(output_file, video_name, score, poly_results, video_len, fps):
    # 息肉个数
    r = stat_ct(poly_results)
    r = merge_ct(r)
#     r = filt_polyp(r)
    poly_num = len(r)
    
    with open(output_file, 'a+') as f:
    # 写息肉出现时间点
        for rr in r:
            f.write('%.3f	%d	%d\n' % (rr[0] / fps, rr[0], rr[1]))
    
    # 写息肉个数和视频长度比
        f.write('%s	%.3f	%.3f\n' % (video_name, score, poly_num/video_len))
        
    return poly_num/video_len
        
# 根据视频图片数量统计视频长度
def get_video_len(video_imgs_dir, fps):
    imgs = list(os.listdir(video_imgs_dir))
    return len(imgs) / (fps * 60)
    

### 挑选真息肉

In [9]:
import shutil
base_dir = r'D:\项目资料\息肉假阳性\20221103'
from_path = os.path.join(base_dir, 'images_crop')
to_path = os.path.join(base_dir, 'images_truepoly_crop')

folder_imgs = [(1149, [(5187, 5740), (6568, 6702)]), (1162, [(1970, 2107), (3923, 4100)]), (1202, [(339, 1036), (4278, 4444)]), 
               (2161, [(912, 996), (2707, 2867), (9939, 10467)]), (1354, [(1633, 2172), (3719, 3847)]),
               (2154, [(2048, 2074), (5192, 5534)]), 
                (1307, [(1131, 1183)]), (1293, [(1303, 1470)]), (1435, [(5616, 5707)]), (2003, [(4532, 4715)]), 
                (2156, [(1254, 1341), (5450, 5872)]), (2195, [(3282, 3664), (4070, 4150), (4718, 4897), (6249, 6354)]), 
               (3513, [(2092, 3350), (4077, 4248)]), (1014, [(5115, 5160), (5591, 5681)]),
                (3538, [(5630, 5721)]), (1447, [(4743, 4879)]), (2135, [(5692, 5782)]), 
               (2230, [(4042, 4153), (4943, 5075), (5095, 5266)]), (3479, [(4690, 4734), (7254, 7987)]),
                (3497, [(1517, 2914), (5764, 5790)])]

for folder, start_ends in folder_imgs:
    from_folder = os.path.join(from_path, str(folder))
    to_folder = os.path.join(to_path, str(folder))
    print('processing: %s' % folder)
    if not os.path.exists(to_folder):
        os.mkdir(to_folder)
        
    for start, end in start_ends:
        for i in range(start, end+1):
            img_name = '%s.jpg' % i
            from_img = os.path.join(from_folder, img_name)
            shutil.copy(from_img, os.path.join(to_folder, img_name))
            if os.path.exists(from_img):
                os.remove(from_img)
              


processing: 1149
processing: 1162
processing: 1202
processing: 2161
processing: 1354
processing: 2154
processing: 1307
processing: 1293
processing: 1435
processing: 2003
processing: 2156
processing: 2195
processing: 3513
processing: 1014
processing: 3538
processing: 1447
processing: 2135
processing: 2230
processing: 3479
processing: 3497


In [22]:
a = [(106, 474), (3211, 3294)]
a[0][1] - a[0][0] + a[1][1] - a[1][0]

451

### 运行息肉检测部分

In [13]:
fps = 18
# score = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35]
base_dir = r'D:\项目资料\息肉假阳性\20221103'

## 假阳性检测
video_names_score=[
#                     ('3419', 0.2),  # 0-5 -> 1.12
#                     ('3429', 0.2),  # 0-5 -> 4.14
#                     ('3434', 0.2),  # 0-5 -> 2.24
#                     ('3435', 0.001), # 15-20 -> 19.11
#                     ('3438', 0.005), # 15-20 -> 15.86
#                     ('3442', 0.002), # 15-20 -> 16.5
#                     ('1149', 0.08),  # 5-10  -> 8.62
#                     ('1162', 0.05),  # 5-10 -> 5.18
#                     ('1202', 0.05),  # 5-10 -> 7.41
#                     ('2161', 0.015),  # 10-15 -> 13.17
#                     ('1354', 0.02),  # 10-15 -> 13.02
#                     ('2154', 0.02),  # 10-15 -> 13.43
#                     ('1307', 0.4), # 0-5 -> 3.58
#                     ('1293', 0.2), # 0-5 -> 3.39
#                     ('1435', 0.5), # 0-5 -> 4.19
#                     ('2003', 0.2), # 0-5 -> 4.95
#                     ('2156', 0.2), # 0-5 -> 3.04
#                     ('2195', 0.8), # 0-5 -> 2.87
#                     ('3513', 0.2), # 0-5 -> 2.76
#                     ('1014', 0.008), # 15-20 -> 18.32
#                     ('3538', 0.002), # 15-20 -> 18.9
#                     ('1447', 0.007), # 15-20 -> 17.56
#                     ('2135', 0.007), # 15-20 -> 19.16
#                     ('2230', 0.005), # 15-20 -> 19.28
#                     ('3479', 0.005), # 15-20 -> 18.73
#                     ('3497', 0.003), # 15-20 -> 15.0
]

# video_names_score=[('1006', 0.015)] # 0.03：8.84, 0.015: 13.95

input_base_dir = os.path.join(base_dir, 'images_crop')
output_base_dir = os.path.join(base_dir, 'images_detect')
for video_name, score in video_names_score:
    print(video_name, score)
    poly_results = detect_polyp_by_model(os.path.join(input_base_dir, video_name), 
                                         os.path.join(output_base_dir, '%s_%s' % (video_name, score)), score)
    val = write_stat_result(os.path.join(base_dir, 'stat.txt'), video_name, score, poly_results,
                      get_video_len(os.path.join(input_base_dir, video_name), fps), fps)
    print(val)
                  

2022-11-05 16:14:16,839 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


3442 0.002
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-11-05 16:28:33,727 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


16.50167117618972
2195 0.8
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-11-05 16:38:48,281 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


2.872849796063132
3497 0.003
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.
15.02692998204668


In [11]:
## 真阳性检测
true_poly_score = 0.001
input_base_dir = os.path.join(base_dir, 'images_truepoly_crop')
output_base_dir = os.path.join(base_dir, 'images_truepoly_detect')
for video_name in os.listdir(input_base_dir):
    score = true_poly_score
    print(video_name, score)
    poly_results = detect_polyp_by_model(os.path.join(input_base_dir, video_name), 
                                         os.path.join(output_base_dir, '%s_%s' % (video_name, score)), score)

2022-11-05 01:37:26,910 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


1014 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-11-05 01:37:47,822 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


1149 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-11-05 01:39:36,361 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


...

3497 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-11-05 01:58:10,334 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


3513 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-11-05 02:01:32,484 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


3538 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


### 运行真息肉标记
暂时不用

In [17]:
input_base_dir = os.path.join(base_dir, 'images_crop')
output_base_dir = os.path.join(base_dir, 'images_detect')

# 找到文件夹对应的score后缀
folder_score = {}
for folder in os.listdir(output_base_dir):
    arr = folder.split('_')
    folder_score[arr[0]] = arr[1]
    
for folder, score in folder_score.items():
    if folder != '3531':
        continue
    print(folder, score)
    results = []
    img_names = get_dir_sorted_imgs(os.path.join(input_base_dir, folder))
    det_img_names = set(get_dir_sorted_imgs(os.path.join(output_base_dir, '%s_%s' % (folder, score))))
    for img_name in img_names:
        if img_name not in det_img_names:
            results.append((img_name, 0))
        else:
            results.append((img_name, 1))
            
    val = write_stat_result(os.path.join(base_dir, 'stat.txt'), folder, float(score), results,
                      get_video_len(os.path.join(input_base_dir, folder), fps), fps)
    print(val)
    

## 假阳性设置score
# for video_name, score in [
#     ('3429', 0.008)
# ]:
## 真阳性检测
# for video_name in os.listdir(input_base_dir):
#     score = true_poly_score
    
#     print(video_name, score)
#     poly_results = detect_polyp_by_model(os.path.join(input_base_dir, video_name), 
#                                          os.path.join(output_base_dir, '%s_%s' % (video_name, score)), score)
#     val = write_stat_result(os.path.join(base_dir, 'stat.txt'), video_name, score, poly_results,
#                       get_video_len(os.path.join(input_base_dir, video_name), fps), fps)
#     print(val)
                  

In [ ]:
### 运行记录
3400 0.01 12.839
3400 0.35 3.959

### 合成视频

In [6]:
import os
import numpy as np
import cv2
fps = 18

# 将生成的息肉图片和原始图片一起合成息肉标记视频
def create_mp4(raw_img_paths, fake_polyp_path, true_polyp_path, output_video_path):
    
    image = cv2.imdecode(np.fromfile(os.path.join(raw_img_paths[0], os.listdir(raw_img_paths[0])[0]), dtype=np.uint8), -1)
    img_h, img_w, _ = image.shape

    video_path = output_video_path
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*"XVID"), fps, (img_w, img_h))
    
    img_names = get_dir_sorted_imgs(raw_img_paths)
    for f in img_names:
        filepath11 = os.path.join(raw_img_paths[0], f)
        filepath12 = os.path.join(raw_img_paths[1], f)
        filepath2 = os.path.join(fake_polyp_path, f)
        filepath3 = os.path.join(true_polyp_path, f)
        if os.path.exists(filepath3):
            filepath = filepath3
        elif os.path.exists(filepath2):
            filepath = filepath2
        elif os.path.exists(filepath11):
            filepath = filepath11
        elif os.path.exists(filepath12):
            filepath = filepath12
        image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8),-1)
        video.write(image)
    video.release()

In [10]:
## 无辅助视频合成
for base_dir in [r'D:\项目资料\息肉假阳性\20221007']:
    raw_img_folders = [os.path.join(base_dir, '..', '20221103', 'images_crop'), os.path.join(base_dir, '..', '20221103', 'images_truepoly_crop')]
    fake_polyp_folder = os.path.join(base_dir, 'images_detect_wfz')
    true_polyp_folder = os.path.join(base_dir, 'images_truepoly_detect_wfz')
    output_video_folder = os.path.join(base_dir, 'videos_gen_wfz')
    
    # 循环处理
    for video_name in os.listdir(os.path.join(base_dir, '..', '20221103', 'images')):
        print('processing: %s' % (video_name))
        raw_img_paths = [os.path.join(rf, video_name) for rf in raw_img_folders]
        fake_polyp_path = os.path.join(fake_polyp_folder, '%s' % (video_name))
        true_polyp_path = os.path.join(true_polyp_folder, '%s' % (video_name))
        output_video_path = os.path.join(output_video_folder, r'%s.avi' % (video_name))

        create_mp4(raw_img_paths, fake_polyp_path, true_polyp_path, output_video_path)

processing: 1014
processing: 1149
processing: 1162
processing: 1202
processing: 1293
processing: 1307
processing: 1354
processing: 1435
processing: 1447
processing: 2003
processing: 2135
processing: 2154
processing: 2156
processing: 2161
processing: 2195
processing: 2230
processing: 3419
processing: 3429
processing: 3434
processing: 3435
processing: 3438
processing: 3442
processing: 3479
processing: 3497
processing: 3513
processing: 3538


In [17]:
for base_dir in [r'D:\项目资料\息肉假阳性\20221103']:
    raw_img_folders = [os.path.join(base_dir, 'images_crop'), os.path.join(base_dir, 'images_truepoly_crop')]
    fake_polyp_folder = os.path.join(base_dir, 'images_detect')
    true_polyp_folder = os.path.join(base_dir, 'images_truepoly_detect')
    output_video_folder = os.path.join(base_dir, 'videos_gen')
    
    # 文件夹名和分数
#     video_names = os.listdir(raw_img_folder)
#     folder_score = {video_name: '' for video_name in video_names}
    folder_score_arr = []
    for folder in os.listdir(fake_polyp_folder):
        arr = folder.split('_')
        folder_score_arr.append((arr[0], arr[1]))
    
    # 循环处理
    for video_name, score in folder_score_arr:
        print('processing: %s, %s' % (video_name, score))
        raw_img_paths = [os.path.join(rf, video_name) for rf in raw_img_folders]
        fake_polyp_path = os.path.join(fake_polyp_folder, '%s_%s' % (video_name, score))
        true_polyp_path = os.path.join(true_polyp_folder, '%s_%s' % (video_name, 0.001))
        output_video_path = os.path.join(output_video_folder, r'%s_%s.avi' % (video_name, score))

        create_mp4(raw_img_paths, fake_polyp_path, true_polyp_path, output_video_path)
        

processing: 1014, 0.008
processing: 1149, 0.08
processing: 1162, 0.05
processing: 1202, 0.05
processing: 1293, 0.2
processing: 1307, 0.4
processing: 1354, 0.02
processing: 1435, 0.5
processing: 1447, 0.007
processing: 2003, 0.2
processing: 2135, 0.007
processing: 2154, 0.02
processing: 2156, 0.2
processing: 2161, 0.015
processing: 2195, 0.8
processing: 2230, 0.005
processing: 3419, 0.2
processing: 3429, 0.2
processing: 3434, 0.2
processing: 3435, 0.001
processing: 3438, 0.005
processing: 3442, 0.002
processing: 3479, 0.005
processing: 3497, 0.003
processing: 3513, 0.2
processing: 3538, 0.002
